In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data= pd.read_csv('..\data\Crash_Data.csv')

C:\Users\Omar\AppData\Local\Temp\ipykernel_21492\1727040306.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('..\data\Crash_Data.csv')


# Data Cleaning

In [3]:
%%writefile functions.py

import pandas as pd
import numpy as np

def column_rename (df: pd.DataFrame) -> pd.DataFrame:
    """
    This function renames column names by removing spaces and converting to lower case
    Inputs: df of type pandas dataframe
    Outputs: returns the dataframe with the renamed columns
    """
    cols =[]
    for x in df.columns:
        if isinstance(x, str):
            cols.append(x.lower().replace(' ', '_'))
        else:
            cols.append(x)
            
    df.columns=cols
    return df



def col_replace_dash (df: pd.DataFrame) -> pd.DataFrame:
    """
    This function renames column names by removing spaces and converting to lower case
    Inputs: df of type pandas dataframe
    Outputs: returns the dataframe with the renamed columns
    """
    cols =[]
    for x in df.columns:
        if isinstance(x, str):
            cols.append(x.lower().replace('-', '_'))
        else:
            cols.append(x)
            
    df.columns=cols
    return df



def clean_agency_name (df: pd.DataFrame) -> pd.DataFrame:
    """
    This function groups the same values of the agency name column
    It uses a dictionary to replace the redundant values
    Inputs: df type pandas dataframe
    Outputs: returns the dataframe with the renamed columns
    """
    new_row_values = {'montgomery county police': 'montgomery', 'rockville police departme' : 'rockville', 'gaithersburg police depar' : 'gaithersburg',
    'takoma park police depart': 'takoma', 'maryland-national capital': 'maryland'}
    df['agency_name']= df['agency_name'].replace(new_row_values)
    return df


def clean_collision_type(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function renames the clean collision type column
    It uses combination of regex and a dictionary to do so
    Inputs: df of type pandas dataframe
    Outputs: returns the dataframe with the renamed values
    """
    replace_short= {'\\bdir\\b' : 'direction', '\\brend\\b' : 'rear end'}
    df['collision_type']= df['collision_type'].replace(replace_short, regex=True)
    return df
    
    
def df_to_lower (df: pd.DataFrame) -> pd.DataFrame:
    """
    This function converts all the values in all the columns to lower case
    Inputs: df of type pandas dataframe
    Outputs: returns the dataframe with the string values in lower case
    """
    df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
    return df


def replace_invalids_of_column(df: pd.DataFrame, num_replacements: float, replacement_value:str, col_to_clean: str) -> pd.DataFrame:
    """
    This function removes the invalid values from the given column
    Inputs: df of type pandas dataframe, the values that will replace the invalid values, and their frequencies
    Outputs: returns dataframe with the replaced invalid values
    """
    invalid_indices= df[df[col_to_clean] == 'invalid'].index
    selected_indices = np.random.choice(invalid_indices, size=num_replacements, replace=False)
    df.loc[selected_indices, col_to_clean] = replacement_value
    return df


Overwriting functions.py


In [4]:
df= data.copy()

In [5]:
%run functions.py

In [6]:
df= col_replace_dash(df)
df= column_rename(df)
df= df_to_lower(df)
df.head()
df.shape

(84278, 44)

In [7]:
df= clean_agency_name(df)
df['agency_name'].value_counts(dropna=False)

agency_name
montgomery      72826
rockville        5171
gaithersburg     3918
takoma           1619
maryland          692
mcpark             52
Name: count, dtype: int64

In [8]:
#after checking the datatypes of the columns, we decided to cast the crash_date/time column to datetime datatype 
df['crash_date/time']=pd.to_datetime(df['crash_date/time'], format="%m/%d/%Y %I:%M:%S %p")
type(df['crash_date/time'])
df['crash_date/time']

0       2019-09-27 09:38:00
1       2021-01-16 21:24:00
2       2021-09-27 15:00:00
3       2022-01-07 19:00:00
4       2020-05-02 14:16:00
                ...        
84273   2022-10-13 14:30:00
84274   2022-10-09 19:45:00
84275   2022-10-17 12:55:00
84276   2022-10-16 11:00:00
84277   2022-09-15 20:00:00
Name: crash_date/time, Length: 84278, dtype: datetime64[ns]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84278 entries, 0 to 84277
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   report_number                 84278 non-null  object        
 1   local_case_number             84278 non-null  object        
 2   agency_name                   84278 non-null  object        
 3   acrs_report_type              84278 non-null  object        
 4   crash_date/time               84278 non-null  datetime64[ns]
 5   hit/run                       84276 non-null  object        
 6   route_type                    73060 non-null  object        
 7   mile_point                    73563 non-null  float64       
 8   mile_point_direction          73071 non-null  object        
 9   lane_direction                73574 non-null  object        
 10  lane_number                   84278 non-null  int64         
 11  lane_type                   

In [10]:
original_null_percentages= df.isnull().sum()/len(df)*100
display(original_null_percentages)

report_number                    0.000000
local_case_number                0.000000
agency_name                      0.000000
acrs_report_type                 0.000000
crash_date/time                  0.000000
hit/run                          0.002373
route_type                      13.310710
mile_point                      12.713876
mile_point_direction            13.297658
lane_direction                  12.700823
lane_number                      0.000000
lane_type                       90.328437
number_of_lanes                  0.000000
direction                       12.713876
distance                        10.500961
distance_unit                   12.702010
road_grade                      13.358172
nontraffic                       0.000000
road_name                       12.702010
cross_street_type               13.329695
cross_street_name               12.707943
off_road_description            87.300363
municipality                    89.280714
related_non_motorist            94

After checking the percentage of missing values for the columns, we dropped the columns with a percentage greater than 20% and also those that were irrelevant to our analysis.
We also made sure to keep the columns that were relevant to our business questions

In [11]:
columns_to_drop=['lane_type', 'off_road_description', 'municipality', 'related_non_motorist', 'non_motorist_substance_abuse',
                 'first_harmful_event', 'second_harmful_event', 'fixed_oject_struck', 'junction', 'intersection_type', 'intersection_area', 
                 'route_type', 'mile_point_direction', 'lane_direction','direction', 'distance_unit' ,'road_name', 'cross_street_type', 'cross_street_name', 
                'surface_condition', 'traffic_control', 'driver_substance_abuse', 'road_alignment' , 'road_division', 'mile_point', 'distance']
df.drop(columns_to_drop, axis=1, inplace= True)

In [12]:
#rechecking the NaNs after dropping the columns with high percentage of missing values
df.isnull().sum()/len(df)*100

report_number         0.000000
local_case_number     0.000000
agency_name           0.000000
acrs_report_type      0.000000
crash_date/time       0.000000
hit/run               0.002373
lane_number           0.000000
number_of_lanes       0.000000
road_grade           13.358172
nontraffic            0.000000
at_fault              0.000000
collision_type        0.510216
weather               8.279741
light                 0.836517
road_condition       16.264031
latitude              0.000000
longitude             0.000000
location              0.000000
dtype: float64

After checking the percentage of NaNs for the remaining columns, the percentages for the following columns were insignificant
We decided to drop the NaNs from these columns.

In [13]:
df=df.dropna(subset=['hit/run', 'collision_type' , 'light'])

In [14]:
df.isnull().sum()/len(df)*100

report_number         0.000000
local_case_number     0.000000
agency_name           0.000000
acrs_report_type      0.000000
crash_date/time       0.000000
hit/run               0.000000
lane_number           0.000000
number_of_lanes       0.000000
road_grade           13.155015
nontraffic            0.000000
at_fault              0.000000
collision_type        0.000000
weather               8.064031
light                 0.000000
road_condition       16.041469
latitude              0.000000
longitude             0.000000
location              0.000000
dtype: float64

In [15]:
df['light'].value_counts(dropna=False)

light
daylight                    54747
dark lights on              20091
dark no lights               3001
dusk                         1904
dawn                         1692
dark -- unknown lighting      929
unknown                       597
other                         186
Name: count, dtype: int64

In [16]:
df['collision_type'].value_counts(dropna=False)

collision_type
same dir rear end               22330
single vehicle                  13914
straight movement angle         12949
other                           11356
same direction sideswipe         7454
head on left turn                5425
head on                          1767
same direction right turn        1664
same direction left turn         1569
opposite direction sideswipe     1360
angle meets left turn             840
unknown                           610
angle meets right turn            522
same dir rend right turn          330
same dir rend left turn           324
angle meets left head on          302
same dir both left turn           290
opposite dir both left turn       141
Name: count, dtype: int64

After checking the value counts for the columns that are significant to our business questions, we decided to drop the NaN values from the road_condition and check its effect on the remaining columns

In [17]:
# Displaying the value counts for road_condition
display(df['road_condition'].value_counts(dropna=False))

road_condition
no defects                  68653
NaN                         13338
other                         310
holes ruts etc                267
unknown                       171
loose surface material        147
foreign material              108
view obstructed                68
shoulder defect                62
obstruction not lighted        14
obstruction not signaled        9
Name: count, dtype: int64

In [18]:
# Replace 'unknown', 'other', and NaN values with 'invalid'
df['road_condition'] = df['road_condition'].replace(['unknown', 'other', np.nan], 'invalid')

# Calculate the sum of 'invalid' values
sum_invalid_values = (df['road_condition'] == 'invalid').sum()

# Displaying the current counts for the categories that are not invalid
unique_values = df['road_condition'][~df['road_condition'].isin(['invalid', 'no defects'])].value_counts()

# Calculate the total count of other values (excluding 'invalid' and 'no defects')
other_values_count = df['road_condition'][~df['road_condition'].isin(['invalid', 'no defects'])].count()

# Finding the amount of invalids to be replaced for each category
distribution_amounts = (unique_values / other_values_count) * sum_invalid_values
distribution_amounts= distribution_amounts.round(0)
distribution_amounts['obstruction not signaled']+=1

# Applying the function to clean the road_condition column and replace the invalid values
for key, value in distribution_amounts.items():
    replace_invalids_of_column(df, int(value), key, 'road_condition')

display(df['road_condition'].value_counts(dropna=False))

road_condition
no defects                  68653
holes ruts etc               5733
loose surface material       3156
foreign material             2319
view obstructed              1460
shoulder defect              1331
obstruction not lighted       301
obstruction not signaled      194
Name: count, dtype: int64

In [19]:
# Displaying the value counts for road_grade
df['road_grade'].value_counts(dropna=False)

road_grade
level             54788
NaN               10938
grade downhill     8656
hill uphill        6401
hill crest         1909
other               169
unknown             117
dip sag             105
on bridge            64
Name: count, dtype: int64

In [20]:
# Replace 'unknown', 'other', and NaN values with 'invalid'
df['road_grade'] = df['road_grade'].replace(['unknown', 'other', np.nan], 'invalid')

# Calculate the sum of 'invalid' values
sum_invalid_values = (df['road_grade'] == 'invalid').sum()

# Displaying the current counts for the categories that are not invalid
unique_values = df['road_grade'][~df['road_grade'].isin(['invalid'])].value_counts()

# Calculate the total count of other values (excluding 'invalid')
other_values_count = df['road_grade'][~df['road_grade'].isin(['invalid'])].count()

# Finding the amount of invalids to be replaced for each category
distribution_amounts = (unique_values / other_values_count) * sum_invalid_values
distribution_amounts= distribution_amounts.round(0)

# Applying the function to clean the road_grade column and replace the invalid values
for key, value in distribution_amounts.items():
    replace_invalids_of_column(df, int(value), key, 'road_grade')

display(df['road_grade'].value_counts(dropna=False))

road_grade
level             63338
grade downhill    10007
hill uphill        7400
hill crest         2207
dip sag             121
on bridge            74
Name: count, dtype: int64

In [21]:
# Displaying the value counts for light
df['light'].value_counts(dropna=False)

light
daylight                    54747
dark lights on              20091
dark no lights               3001
dusk                         1904
dawn                         1692
dark -- unknown lighting      929
unknown                       597
other                         186
Name: count, dtype: int64

In [22]:
# Displaying the value counts for weather
df['weather'].value_counts(dropna=False)

weather
clear                       55580
raining                     10012
cloudy                       8416
NaN                          6705
snow                          853
unknown                       539
foggy                         342
wintry mix                    241
other                         182
sleet                         119
severe winds                   84
blowing snow                   67
blowing sand, soil, dirt        7
Name: count, dtype: int64

In [23]:
# Replace 'unknown', 'other', and NaN values with 'invalid'
df['weather'] = df['weather'].replace(['unknown', 'other', np.nan], 'invalid')

# Calculate the sum of 'invalid' values
sum_invalid_values = (df['weather'] == 'invalid').sum()

# Displaying the current counts for the categories that are not invalid
unique_values = df['weather'][~df['weather'].isin(['invalid'])].value_counts()

# Calculate the total count of other values (excluding 'invalid')
other_values_count = df['weather'][~df['weather'].isin(['invalid'])].count()

# Finding the amount of invalids to be replaced for each category
distribution_amounts = (unique_values / other_values_count) * sum_invalid_values
distribution_amounts= distribution_amounts.round(0)
distribution_amounts['clear']-=2

# Applying the function to clean the weather column and replace the invalid values
for key, value in distribution_amounts.items():
    replace_invalids_of_column(df, int(value), key, 'weather')

display(df['weather'].value_counts(dropna=False))

weather
clear                       61029
raining                     10994
cloudy                       9241
snow                          937
foggy                         376
wintry mix                    265
sleet                         131
severe winds                   92
blowing snow                   74
blowing sand, soil, dirt        8
Name: count, dtype: int64

In [24]:
df= clean_collision_type(df)
display(df['collision_type'].value_counts(dropna=False))

collision_type
same direction rear end               22330
single vehicle                        13914
straight movement angle               12949
other                                 11356
same direction sideswipe               7454
head on left turn                      5425
head on                                1767
same direction right turn              1664
same direction left turn               1569
opposite direction sideswipe           1360
angle meets left turn                   840
unknown                                 610
angle meets right turn                  522
same direction rear end right turn      330
same direction rear end left turn       324
angle meets left head on                302
same direction both left turn           290
opposite direction both left turn       141
Name: count, dtype: int64

In [25]:
# Replace 'unknown' and 'other' with 'invalid'
df['collision_type'] = df['collision_type'].replace(['unknown', 'other'], 'invalid')

# Calculate the sum of 'invalid' values
sum_invalid_values = (df['collision_type'] == 'invalid').sum()

# Displaying the current counts for the categories that are not invalid
unique_values = df['collision_type'][~df['collision_type'].isin(['invalid'])].value_counts()

# Calculate the total count of other values (excluding 'invalid')
other_values_count = df['collision_type'][~df['collision_type'].isin(['invalid'])].count()


# Finding the amount of invalids to be replaced for each category
distribution_amounts = (unique_values / other_values_count) * sum_invalid_values
distribution_amounts= distribution_amounts.round(0)

distribution_amounts['same direction rear end']-=1

# Applying the function to clean the collision_type column and replace the invalid values
for key, value in distribution_amounts.items():
    replace_invalids_of_column(df, int(value), key, 'collision_type')

display(df['collision_type'].value_counts(dropna=False))

collision_type
same direction rear end               26083
single vehicle                        16253
straight movement angle               15126
same direction sideswipe               8707
head on left turn                      6337
head on                                2064
same direction right turn              1944
same direction left turn               1833
opposite direction sideswipe           1589
angle meets left turn                   981
angle meets right turn                  610
same direction rear end right turn      385
same direction rear end left turn       378
angle meets left head on                353
same direction both left turn           339
opposite direction both left turn       165
Name: count, dtype: int64

In [26]:
# final check 
df.isnull().sum()/len(df)*100

report_number        0.0
local_case_number    0.0
agency_name          0.0
acrs_report_type     0.0
crash_date/time      0.0
hit/run              0.0
lane_number          0.0
number_of_lanes      0.0
road_grade           0.0
nontraffic           0.0
at_fault             0.0
collision_type       0.0
weather              0.0
light                0.0
road_condition       0.0
latitude             0.0
longitude            0.0
location             0.0
dtype: float64

In [27]:
df.to_csv('..\data\clean_crash_data.csv',index = False)

In [28]:
!dir

 Volume in drive D has no label.
 Volume Serial Number is 089C-0519

 Directory of D:\Ironhack\Classwork\Week_4\Day_5\project_folder\notebooks

12/22/2023  04:33 AM    <DIR>          .
12/22/2023  04:33 AM    <DIR>          ..
12/22/2023  03:15 AM    <DIR>          .ipynb_checkpoints
12/22/2023  04:33 AM            15,876 Data_Agg_and_EDA.ipynb
12/21/2023  03:12 PM            18,798 data_cleaning_and_formatting.ipynb
12/22/2023  08:31 AM             3,116 functions.py
12/22/2023  04:28 AM             2,785 hypothesis_testing.ipynb
12/21/2023  02:47 PM    <DIR>          __pycache__
               4 File(s)         40,575 bytes
               4 Dir(s)  525,151,133,696 bytes free


In [29]:
df_clean= pd.read_csv('..\data\clean_crash_data.csv')
display(df_clean)

,report_number,local_case_number,agency_name,acrs_report_type,crash_date/time,hit/run,lane_number,number_of_lanes,road_grade,nontraffic,at_fault,collision_type,weather,light,road_condition,latitude,longitude,location
0,mcp3048005t,190046316,montgomery,property damage crash,2019-09-27 09:38:00,no,0,0,level,yes,driver,single vehicle,clear,daylight,foreign material,39.026700,-77.136785,"(39.0267, -77.136785)"
1,dm8376000z,210002024,takoma,property damage crash,2021-01-16 21:24:00,no,0,2,level,no,driver,same direction sideswipe,clear,dark lights on,no defects,38.979030,-77.015981,"(38.97903039, -77.01598148)"
2,mcp12050078,210038391,montgomery,property damage crash,2021-09-27 15:00:00,yes,0,0,grade downhill,yes,driver,straight movement angle,clear,daylight,holes ruts etc,39.023680,-77.145418,"(39.02367954, -77.14541849)"
3,mcp003600g5,220000831,montgomery,property damage crash,2022-01-07 19:00:00,no,0,0,level,yes,unknown,single vehicle,clear,dark lights on,holes ruts etc,39.058047,-77.125307,"(39.058047, -77.125307)"
4,ej78460055,200017938,gaithersburg,property damage crash,2020-05-02 14:16:00,yes,0,0,level,yes,driver,straight movement angle,clear,daylight,holes ruts etc,39.147956,-77.229616,"(39.14795617, -77.2296164)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83142,ha24980004,22001814,maryland,property damage crash,2022-10-13 14:30:00,yes,0,1,level,no,driver,single vehicle,raining,daylight,no defects,39.212050,-77.281670,"(39.2120499, -77.2816699)"
83143,mcp2708007t,220044172,montgomery,property damage crash,2022-10-09 19:45:00,yes,3,3,level,no,driver,same direction rear end,clear,dark lights on,no defects,39.046391,-76.986974,"(39.04639108, -76.98697395)"
83144,mcp21950059,220045530,montgomery,property damage crash,2022-10-17 12:55:00,yes,0,1,level,no,driver,same direction rear end,cloudy,daylight,no defects,39.104377,-77.157032,"(39.10437667, -77.15703167)"
83145,mcp2592002g,220045353,montgomery,injury crash,2022-10-16 11:00:00,no,1,1,level,no,driver,single vehicle,clear,daylight,loose surface material,39.207037,-77.182050,"(39.20703667, -77.18205)"


In [30]:
df.describe()

,crash_date/time,lane_number,number_of_lanes,latitude,longitude
count,83147,83147.000000,83147.000000,83147.000000,83147.000000
mean,2018-09-10 19:39:11.172020480,1.212263,2.060928,39.084078,-77.114409
min,2015-01-01 00:30:00,0.000000,0.000000,37.720000,-79.486000
25%,2016-10-21 07:31:30,1.000000,1.000000,39.025289,-77.191313
50%,2018-07-27 15:24:00,1.000000,2.000000,39.075877,-77.107518
75%,2020-07-03 19:01:30,2.000000,3.000000,39.140613,-77.040659
max,2022-10-20 14:21:00,8.000000,99.000000,39.990414,-75.527708
std,NaN,0.977662,1.344392,0.073062,0.100115
